In [ ]:
import json
import pandas as pd 
import numpy as np 
df1=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df2=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
df2.drop(['homepage'], axis=1,inplace=True)

In [ ]:
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')
df2.set_index('id',inplace=True)

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_author(x):
    for i in x:
        if i['job'] == 'Author':
            return i['name']
    return np.nan

In [ ]:
df2.columns

In [ ]:
df2['crew'] = df2['crew'].apply(json.loads)
df2['genres'] = df2['genres'].apply(json.loads)
df2['keywords'] = df2['keywords'].apply(json.loads)
df2['production_companies'] = df2['production_companies'].apply(json.loads)
df2['production_countries'] = df2['production_countries'].apply(json.loads)
df2['cast'] = df2['cast'].apply(json.loads)


df2['director'] = df2['crew'].apply(get_director)
df2['author'] = df2['crew'].apply(get_author)

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        return names
    
        if len(names) > 3:
            names = names[:3]
        return names
    
    return []

In [ ]:
df2['keywords_list'] = df2['keywords'].apply(get_list)
df2['genres_list'] = df2['genres'].apply(get_list)
df2['companies_list'] = df2['production_companies'].apply(get_list)
df2['countries_list'] = df2['production_countries'].apply(get_list)
df2['cast_list'] = df2['cast'].apply(get_list)

df2['cast_list'] = df2['cast_list'].apply(lambda x : x[:3])

In [ ]:
df2.info()

In [ ]:
df2.head()

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
features = ['keywords_list','genres_list','companies_list','countries_list','cast_list','director']
for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
df2.drop(['cast','production_companies','production_countries','keywords','genres','original_title','crew'],axis=1,inplace=True)

In [ ]:
df2.columns

In [ ]:
df_preprocess=df2[['budget','overview','title','director','author','keywords_list','genres_list','companies_list','countries_list','cast_list']]

In [ ]:
df_preprocess.head()

In [ ]:
df_preprocess['author'] = df_preprocess['author'].apply(lambda x : x if str(x) != 'nan' else "")

In [ ]:
def create_soup(x,pesos=[1,1,1,1,0]):
    return ' '.join(pesos[1]*x['keywords_list']) + ' ' + \
    ' '.join(pesos[2]*x['genres_list']) + ' ' + ' '.join(pesos[3]*x['cast_list']) + ' '.join(pesos[4]*x['companies_list']) + " "+ pesos[0] * (x['director'] + ' ')

df_preprocess['soup'] = df_preprocess.apply(create_soup, axis=1)

In [ ]:
df_preprocess['soup'] = df_preprocess['soup'].apply(lambda x : " ".join(x.split()))

In [ ]:
df_preprocess[df_preprocess['title']=='The Godfather']['cast_list'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_preprocess['soup'])

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df_preprocess = df_preprocess.reset_index()
indices = pd.Series(df_preprocess.index, index=df2['title'])

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df_preprocess['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Godfather')

In [ ]:
get_recommendations('The Dark Knight Rises')

In [ ]:
get_recommendations('Avatar')

In [ ]:
get_recommendations('Pulp Fiction')

In [ ]:
get_recommendations('Click')

- Considerar features numéricas (budget e lancamento)
- Considerar faixa etária (desenho animado ta ruim)
- Acha filmes bem similares muito bem


Peso 1

Peso 21120,21210

In [ ]:
get_recommendations('Toy Story 2')